<a href="https://colab.research.google.com/github/jonesavelino/idea-tool/blob/main/IDEA_ETAPA_2_ConverterDoccanoSpacy3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETAPA 2 - CONVERTER O TEXTOS ANOTADOS PELO DOCCANO

## Caderno resposável por receber o arquivo exportado do Doccano, no padrão JSON Lines, com as entidades (entities) e relações (relations) destacadas por posição no arquivo.

### O objetivo é transformar esse arquivo em JSON para SPACY

### Pré-requisitos:

#### 1- Arquivo gerado pelo Doccano
#### 2- Montagem do Drive

### Parâmetros:

#### DATA_FILE_PATH_IN = Arquivo gerado pelo Doccano (/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/inputs/exporta_anotacao_DD_MM_YYYY.jsonl)

#### DATA_FILE_PATH_OUT = Arquivo de Saída no padrão do SPACY (/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao_17_05_2023.spacy)
--
# Data: 18-05-2023
# Autores: Maj Giselle Farias e Jones.

Fonte: https://www.kaggle.com/code/abneetwats24/doccano-to-spacy3-data-conversion-and-training/notebook

### 1 - Montagem do Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2 - Imports e bibliotecas

In [ ]:
#!pip uninstall spacy -y

In [2]:
#!pip install -U spacy==3.1.0
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.


In [3]:
!pip install -U allennlp

# para utilizar o BERT Base
!python -m spacy download pt_core_news_sm

# para utilizar o BERT Large
#!python -m spacy download pt_core_news_lg

# para utilizar o BERT em Inglês Base
#!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 35.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 113.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 121.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [4]:
# Trecho do código "Auxiliares"

!pip install jsonlines

import os
import gzip
import json
import random
import math
import jsonlines


def pega_todos_ids():
    # folder path
    dir_path = r'/Users/gisellerosa/Downloads/sciERC_raw_data'

    # list to store files
    res = []
    res_ann = []

    # Iterate directory
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            res.append(path)
        if(path.endswith('.ann')):
            res_ann.append(path[:-4])
    #print(res_ann)

    return res_ann


def dicts_to_jsonl(data: list, filename: str, compress: bool = True) -> None:
    """
    Method saves list of dicts into jsonl file.

    :param data: (list) list of dicts to be stored,
    :param filename: (str) path to the output file. If suffix .jsonl is not given then methods appends
        .jsonl suffix into the file.
    :param compress: (bool) should file be compressed into a gzip archive?
    """

    sjsonl = '.jsonl'
    sgz = '.gz'

    # Check filename

    if not filename.endswith(sjsonl):
        filename = filename + sjsonl

    # Save data

    if compress:
        filename = filename + sgz
        with gzip.open(filename, 'w') as compressed:
            for ddict in data:
                jout = json.dumps(ddict) + '\n'
                jout = jout.encode('utf-8')
                compressed.write(jout)
    else:
        with open(filename, 'w') as out:
            for ddict in data:
                jout = json.dumps(ddict) + '\n'
                out.write(jout)


def valida_start(texto, start):
    if texto[start] != ' ':
        pos = start - 1
        while texto[pos] != ' ':
            pos -= 1
    else:
        pos = start
        while texto[pos] == ' ':
            pos += 1
    return pos


def valida_end(texto, end):
    if texto[end] == ' ':
        pos = end - 1
        while pos == ' ':
            pos -= 1
        return pos
    else:
        pos = end - 1
        if (pos + 1) >= len(texto):
            pos = end
        elif texto[end + 1] == ' ':
                pos = end
        while texto[pos + 1] != ' ':
            pos -= 1
        return pos



def valida_end2(texto, end):
    texto = texto + '  '
    if not texto[end].isalnum():
        if texto[end-1].isalnum():
            return end-1
        else:
            end -= 1
            while not texto[end].isalnum():
                end -= 1
            return end
    else:
        if texto[end-1].isalnum():
            pos = end
            while texto[pos + 1].isalnum():
                pos += 1
            return pos - 1
        else:
            end -= 1
            while texto[end].isalnum():
                end -= 1
                if texto[end - 1].isalnum():
                    return end - 1
                else:
                    end -= 1
                    while not texto[end].isalnum():
                        end -= 1
                    return end
            return end


def train_test_split_total(data_file_path, percent_split, split1_path, split2_path):
    with open(data_file_path, 'r') as jsonl_file:
        json_total_list = []
        num_lines = 0

        for line in jsonl_file:
            json_total_list.append(json.loads(line))
            num_lines += 1

        print('Quantidade de linhas:', num_lines)

        qtd_split = math.floor(percent_split * num_lines)

        print('qtd_split_train:', qtd_split)

        randomlist = random.sample(range(0, num_lines), qtd_split)

        json_1_list = []
        json_2_list = []

        for line_num in range(num_lines):
            if line_num in randomlist:
                json_1_list.append(json_total_list[line_num])
            else:
                json_2_list.append(json_total_list[line_num])

        print('tamanho json_train_list:', len(json_1_list))
        print('tamanho json_dev_list:', len(json_2_list))

    with open(split1_path, 'w') as f_saida:
        for js in json_1_list:
            jout = json.dumps(js) + '\n'
            f_saida.write(jout)

    with open(split2_path, 'w') as f_saida_2:
        for js in json_2_list:
            jout = json.dumps(js) + '\n'
            f_saida_2.write(jout)

In [9]:
import json
import os
import random
import logging
# from sklearn.metrics import classification_report
# from sklearn.metrics import precision_recall_fscore_support
# # from spacy.gold import GoldParse
# # from spacy.scorer import Scorer
# from sklearn.metrics import accuracy_score
import spacy

print(spacy.__version__)
import re
import random

# spacy.prefer_gpu()

# DATA_FILE_PATH_IN = "inputs/scierc-test.jsonl"
# DATA_FILE_PATH_OUT = "outputs/scierc-test.spacy"
#exporta_doccano_02_10_2023

DATA_FILE_PATH_IN = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/inputs/admin.jsonl"
DATA_FILE_PATH_OUT = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao.spacy"

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for id_ent, label, start, end in entities:
            # print('analisando id: ', id)
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label, id_ent])
        cleaned_data.append([text, {'entities': valid_entities, 'relations': annotations['relations']}])
    return cleaned_data


def validate_overlap(ALL_DATA):
    for ix, x in enumerate(ALL_DATA):
        startCK = []
        for iy, y in enumerate(x[-1]['entities']):
            if iy == 0:
                startCK.append([y[0], y[1]])
            else:
                pop = False
                for z in startCK:
                    if z[0] <= y[0] < z[1]:
                        print(y, z)
                        ALL_DATA[ix][-1]['entities'].pop(iy)
                        print(ALL_DATA[ix][-1]['entities'].pop(iy))
                        pop = True
                        break
                if pop == False:
                    startCK.append([y[0], y[1]])
    return ALL_DATA


def convert_doccano_to_spacy(doccano_JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(doccano_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['text']
            entities = data['entities']
            relations = data['relations']
            if len(entities) > 0:
                for i, entity in enumerate(entities):
                    entities[i] = list(entity.values())
                for j, relation in enumerate(relations):
                    relations[j] = list(relation.values())
                training_data.append((text, {"entities": entities, "relations": relations}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None


def conversao_jsonl_spacy(data_file_path_in, data_file_path_out):
    ALL_DATA = convert_doccano_to_spacy(data_file_path_in)

    ALL_DATA = trim_entity_spans(ALL_DATA)
    ALL_DATA = validate_overlap(ALL_DATA)
    random.shuffle(ALL_DATA)

    #print("RESULTADO FINAL")
    #print(ALL_DATA)

    import pandas as pd
    from tqdm import tqdm
    import spacy
    from spacy.tokens import DocBin, Doc
    from wasabi import Printer

#*********************************************************************************************************************************************
# CONFIGURAÇÃO DOS RÓTULOS/CATEGORIAS DE RELATIONS UTILIZADOS NA ANOTAÇÃO LÁ NO DOCCANO
# E A LÍNGUA UTILIZADA PARA GERAR O ARQUIVO .SPACY (PT - PARA PORTUGUÊS)
#*********************************************************************************************************************************************

    nlp = spacy.blank("pt")  # load a new pipeline spacy model com o vocabulário em português
    MAP_LABELS = {
        "composed_of": "composed_of",
        "instance_of": "instance_of",
        "applied_to": "applied_to",
        "occurs_in": "occurs_in",
        "co-referenced": "co-referenced",
        "responsible_for": "responsible_for",
        "capacity_of": "capacity_of",
        "type_of": "type_of",
        "defined_by": "defined_by",
        "capacity_of": "capacity_of",
        "equivalent_to": "equivalent_to",
        "associated_with": "associated_with"
    }

#*********************************************************************************************************************************************
# CONFIGURAÇÃO DOS RÓTULOS/CATEGORIAS DE RELATIONS UTILIZADOS NA ANOTAÇÃO LÁ NO DOCCANO
# E A LÍNGUA UTILIZADA PARA GERAR O ARQUIVO .SPACY (EN - PARA INGLÊS) - CORPUS QUÍMICO
#*********************************************************************************************************************************************

    # nlp = spacy.blank("en")  # load a new pipeline spacy model com o vocabulário em inglês
    # MAP_LABELS = {
    #     "coexiste_com": "coexiste_com",
    #     "caract_por": "caract_por",
    #     "feito_de": "feito_de",
    #     "sintetizado_por": "sintetizado_por",
    #     "usado_para": "usado_para",
    #     "complemento_de": "complemento_de"
    # }


#*********************************************************************************************************************************************
# CONFIGURAÇÃO DOS RÓTULOS/CATEGORIAS DE RELATIONS UTILIZADOS NA ANOTAÇÃO LÁ NO DOCCANO
# E A LÍNGUA UTILIZADA PARA GERAR O ARQUIVO .SPACY (EN - PARA INGLÊS) - SCIERC
#*********************************************************************************************************************************************

    # nlp = spacy.blank("en")  # load a new pipeline spacy model com o vocabulário em inglês
    # MAP_LABELS = {
    #     "USED-FOR": "USED-FOR",
    #     "CONJUNCTION": "CONJUNCTION",
    #     "COREF": "COREF",
    #     "HYPONYM-OF": "HYPONYM-OF",
    #     "FEATURE-OF": "FEATURE-OF",
    #     "EVALUATE-FOR": "EVALUATE-FOR",
    #     "COMPARE": "COMPARE",
    #     "PART-OF": "PART-OF"
    #  }


    Doc.set_extension("rel", default={}, force=True)
    db = DocBin(store_user_data=True)  # create a DocBin object
    # db = DocBin(docs=docs["total"], store_user_data=True) ???????????????

    msg = Printer()

    c = 0
    for text, annot in tqdm(ALL_DATA):  # data in previous format
        doc = nlp.make_doc(text)  # create doc object from text
        ents = []
        span_starts = {}
        neg = 0
        pos = 0
        print('\n\n-----------------')
        print('-----------------\nTEXT:', text)
        for start, end, label, id_ent in annot["entities"]:  # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
            if span is None:
                #             print("======================================================Skipping entity Start===================================================")
                #             print(start, end, label, span)
                #             print(doc.text[0:end-1],doc.text[start],doc.text[end],'kh',sep='|')
                #             print("======================================================Skipping entity End===================================================")
                print('------------------------Erro de span------------------------')
                print('\t', 'id_ent:', id_ent)
                print('\t', 'label:', label)
                print('\t', 'start:', start)
                print('\t', 'end:', end)
                print('\t', 'text:', text)
                print('\t', 'span:', c)
                print('------------------------------------------------------------')
                c += 1
                s = doc.text
                sub_E = s[end:]
                sub_S = s[:start]
                end = end + (0 if len(sub_E.split(" ", 1)[0]) <= 0 else len(sub_E.split(" ", 1)[0]))
                start = start - (0 if len(sub_S.rsplit(" ", 1)[-1]) <= 0 else len(sub_S.rsplit(" ", 1)[-1]))
                #             print(s[start:end])
                span = doc.char_span(start, end, label=label, alignment_mode="strict")
                if span is None:
                    print("++++++++++++++++++++++++++++Skipping entity Start++++++++++++++++++++++++++++")
                    print(start, end, label, span)
                    print(doc.text[start:end], doc.text[start], doc.text[end - 1], 'kh', sep='|')
                    print("++++++++++++++++++++++++++++Skipping entity End++++++++++++++++++++++++++++++")
                    c += 1
            else:
                ents.append(span)
                span_starts[
                    id_ent] = span.start  # Criando dict com informacao de id da entidade e token inicial da entidade

        doc.ents = ents  # label the text with the ents
        # spacy.displacy.render(doc, style="ent")

        # CONVERSAO DAS RELACOES

        # Criando lista com  todas as entidades
        all_starts = []
        rels = {}
        for e in doc.ents:
            all_starts.append(e.start)

        # print(all_starts)

        for x1 in all_starts:
            for x2 in all_starts:
                rels[(x1, x2)] = {}

        relations = annot["relations"]
        # [{"id":10,"from_id":52,"to_id":51,"type":"usada_para"}]
        print("len(relations):", len(relations))

        for relation in relations:
            from_id = relation[1]  # "from_id"
            start = span_starts[from_id]
            to_id = relation[2]  # "to_id"
            end = span_starts[to_id]
            label = relation[3]  # "type"
            # print('\tstart/end: ', start, end)
            # print("rels[(start, end)]:", [(start, end)])
            # label = MAP_LABELS[label]
            if label not in rels[(start, end)]:
                rels[(start, end)][label] = 1.0
                pos += 1

        # The annotation is complete, so fill in zero's where the data is missing
        for x1 in all_starts:
            for x2 in all_starts:
                for label in MAP_LABELS.values():
                    if label not in rels[(x1, x2)]:
                        neg += 1
                        rels[(x1, x2)][label] = 0.0
                        #print(rels[(x1, x2)])

        doc._.rel = rels

        print('\n')
        print('doc._.rel:')
        print(doc._.rel)

        db.add(doc)

    # #print(len(docs["total"]))
    #
    # # msg.info(
    # #     f"{len(docs['total'])} training sentences"
    # # )

    db.to_disk(data_file_path_out)  # save the docbin object
    print('erros de spans:', c)

3.3.3


In [12]:
#from auxiliares import train_test_split_total
#from conversao_json_spacy_completo import conversao_jsonl_spacy

# Nesse trecho de código foi inserida a Validação Cruzada para Entidades

TOTAL_JSONL_PATH = DATA_FILE_PATH_IN
PERCENT_SPLIT_TEST = 0.8
PERCENT_SPLIT_DEV = 0.2

#Variável que permite dividir o conjunto de dados por X pedaços.
# Isto é, se tiver um arquivo com 1000 registros,
# e a varíavel estiver "setada" para 4, serão gerados 4 arquivos com 250 registros em cada.
NR_FOLDS = 1

# São divididos o conjunto de dados em 3 partes (teste, treino e dev)
TEST_PATH = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-test"
TRAIN_PATH =  "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-train"
DEV_PATH = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-dev"

# dev_jsonl_path = DEV_PATH + ".jsonl"
# dev_spacy_path = DEV_PATH + ".spacy"
# train_jsonl_path = TRAIN_PATH + ".jsonl"
# train_spacy_path = TRAIN_PATH + ".spacy"

test_jsonl_path = TEST_PATH + ".jsonl"
test_spacy_path = TEST_PATH + ".spacy"


# Cria um arquivo temporário que não é utilizado
temp_path = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-temp.jsonl"

# Separando conjunto de teste dos demais (temp)
train_test_split_total(TOTAL_JSONL_PATH, PERCENT_SPLIT_TEST, temp_path, test_jsonl_path)

#REMOVIDO PARA FACILITAR DEPOIS TEM QUE SER REATIVADO
#train_test_split_total(TOTAL_JSONL_PATH, PERCENT_SPLIT_TEST, train_jsonl_path, test_jsonl_path)

# Convertendo jsonl para spacy  (conjunto de teste)
#conversao_jsonl_spacy(train_jsonl_path, train_spacy_path)

conversao_jsonl_spacy(test_jsonl_path, test_spacy_path)
####valida_conversao_jsonl_spacy(test_jsonl_path, test_spacy_path)

#print(train_jsonl_path)
#print(test_jsonl_path)

# Criando pares de conjuntos train+dev para cada fold
for fold in range(NR_FOLDS):
    print(fold)

    train_jsonl_path_k = TRAIN_PATH + str(fold) + ".jsonl"
    train_spacy_path_k = TRAIN_PATH + str(fold) + ".spacy"
    dev_jsonl_path_k = DEV_PATH + str(fold) + ".jsonl"
    dev_spacy_path_k = DEV_PATH + str(fold) + ".spacy"

    # Separando conjuntos train e dev
    train_test_split_total(temp_path, PERCENT_SPLIT_DEV, train_jsonl_path_k, dev_jsonl_path_k)

    # Convertendo jsonl para spacy  (conjunto train)
    conversao_jsonl_spacy(train_jsonl_path_k, train_spacy_path_k)

    # Convertendo jsonl para spacy  (conjunto dev)
    conversao_jsonl_spacy(dev_jsonl_path_k, dev_spacy_path_k)

Quantidade de linhas: 101
qtd_split_train: 80
tamanho json_train_list: 80
tamanho json_dev_list: 21


 14%|█▍        | 3/21 [00:00<00:00, 29.13it/s]



-----------------
-----------------
TEXT: DEMOLIÇÃO – Consiste nos trabalhos para desfazer ou destruir um recurso físico.  DEMONSTRAÇÃO – Ação destinada a iludir o inimigo, por meio de uma exibição de força, em área onde não se procura obter uma decisão, sem, contudo, haver o contato. Ver AÇÃO DIVERSIONÁRIA e FINTA.  DEMONSTRAÇÃO ANFÍBIA – Modalidade de operação anfíbia que compreende a aproximação do território inimigo por Forças Navais, inclusive com meios que caracterizam uma operação anfíbia, sem o efetivo desembarque de tropa. Pode ter como propósito confundir o inimigo quanto ao local da operação principal ou induzi-lo a empreender ações que nos sejam favoráveis.  DEMONSTRAÇÃO DE FORÇA – Ação destinada a dissuadir, iludir ou intimidar as forças adversas por meio da exibição de força.  DEPÓSITO – Instalação de suprimento destinada, basicamente, à estocagem do nível de suprimento prescrito ao escalão que o opera, para distribuição aos elementos a apoiar.  DEPÓSITO ESPECIALIZADO –

 29%|██▊       | 6/21 [00:00<00:00, 28.73it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 70%|███████   | 45/64 [00:01<00:00, 30.18it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 77%|███████▋  | 49/64 [00:02<00:00, 27.69it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate

erros de spans: 0


In [ ]:
# def valida_conversao_jsonl_spacy(data_file_path_in, data_file_path_out):
#     ALL_DATA = convert_doccano_to_spacy(data_file_path_in)

#     ALL_DATA = trim_entity_spans(ALL_DATA)
#     ALL_DATA = validate_overlap(ALL_DATA)
#     random.shuffle(ALL_DATA)
#     vet_erros = []

#     import pandas as pd
#     from tqdm import tqdm
#     import spacy
#     from spacy.tokens import DocBin, Doc
#     from wasabi import Printer

#     MAP_LABELS = {
#         "composed_of": "composed_of",
#         "instance_of": "instance_of",
#         "applied_to": "applied_to",
#         "occurs_in": "occurs_in",
#         "co-referenced": "co-referenced",
#         "responsible_for": "responsible_for",
#         "capacity_of": "capacity_of",
#         "type_of": "type_of",
#         "defined_by": "defined_by",
#         "capacity_of": "capacity_of",
#         "equivalent_to": "equivalent_to",
#         "associated_with": "associated_with"
#     }

#     nlp = spacy.blank("pt")  # load a new pipeline spacy model com o vocabulário em português
#     Doc.set_extension("rel", default={}, force=True)
#     db = DocBin(store_user_data=True)  # create a DocBin object
#     # db = DocBin(docs=docs["total"], store_user_data=True) ???????????????

#     msg = Printer()

#     c = 0
#     for text, annot in tqdm(ALL_DATA):  # data in previous format
#         doc = nlp.make_doc(text)  # create doc object from text
#         ents = []
#         span_starts = {}
#         neg = 0
#         pos = 0
#         print('\n\n-----------------')
#         print('-----------------\nTEXT:', text)
#         for start, end, label, id_ent in annot["entities"]:  # add character indexes
#             span = doc.char_span(start, end, label=label, alignment_mode="strict")
#             if span is None:
#                 print('------------------------Erro de span------------------------')
#                 print('\t', 'id_ent:', id_ent)
#                 print('\t', 'label:', label)
#                 print('\t', 'start:', start)
#                 print('\t', 'end:', end)
#                 print('\t', 'text:', text)
#                 print('------------------------------------------------------------')
#                 vet_erros.append(id_ent)
#                 c += 1
#                 s = doc.text
#                 print('erros de spans:', c)
#         for i in vet_erros:
#           print(i)

In [ ]:
# TOTAL_JSONL_PATH = DATA_FILE_PATH_IN
# PERCENT_SPLIT_TEST = 0.6
# TEST_PATH = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-test"
# NR_FOLDS = 1
# test_jsonl_path = TEST_PATH + ".jsonl"
# test_spacy_path = TEST_PATH + ".spacy"
# temp_path = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-temp.jsonl"
# train_test_split_total(TOTAL_JSONL_PATH, PERCENT_SPLIT_TEST, temp_path, test_jsonl_path)
# conversao_jsonl_spacy(test_jsonl_path, test_spacy_path)


In [ ]:
# TOTAL_JSONL_PATH = DATA_FILE_PATH_IN
# PERCENT_SPLIT_TEST = 0.8
# PERCENT_SPLIT_DEV = 0.2

# # São divididos o conjunto de dados em 3 partes (teste, treino e dev)
# TEST_PATH = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-test"
# TRAIN_PATH =  "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-train"
# DEV_PATH = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-dev"

# test_jsonl_path = TEST_PATH + ".jsonl"
# test_spacy_path = TEST_PATH + ".spacy"

# # Cria um arquivo temporário que não é utilizado
# temp_path = "/content/drive/MyDrive/Doutorado/IDEA/dataset_doccano/outputs/exporta_anotacao-temp.jsonl"

# valida_conversao_jsonl_spacy(test_jsonl_path, test_spacy_path)

In [ ]:
# data_file_path=DATA_FILE_PATH_IN
# percent_split = 0.8

# with open(data_file_path, 'r') as jsonl_file:
#         json_total_list = []
#         num_lines = 0

#         for line in jsonl_file:
#             json_total_list.append(json.loads(line))
#             num_lines += 1

#         print('Quantidade de linhas:', num_lines)

#         qtd_split = math.floor(percent_split * num_lines)

#         print('qtd_split_train:', qtd_split)

#         randomlist = random.sample(range(0, num_lines), qtd_split)

#         json_1_list = []
#         json_2_list = []

#         for line_num in range(num_lines):
#             if line_num in randomlist:
#                 json_1_list.append(json_total_list[line_num])
#             else:
#                 json_2_list.append(json_total_list[line_num])

#         print('tamanho json_train_list:', len(json_1_list))
#         print('tamanho json_dev_list:', len(json_2_list))


Quantidade de linhas: 1
qtd_split_train: 0
tamanho json_train_list: 0
tamanho json_dev_list: 1
